# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kerikeri,-35.2268,173.9474,52.92,84,6,8.99,NZ,1715199581
1,1,aden,12.7794,45.0367,86.97,73,92,7.92,YE,1715199581
2,2,miragoane,18.4423,-73.0876,84.36,68,41,5.91,HT,1715199581
3,3,port-aux-francais,-49.3500,70.2167,40.44,74,90,18.97,TF,1715199581
4,4,lichinga,-13.3128,35.2406,64.49,72,0,9.22,MZ,1715199582


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo=True,
    frame_width=800,
    frame_height=600,
    hover_cols=["City"], 
    xlim=(-180, 180),  
    ylim=(-90, 90),
    tiles='CartoLight',
    color = "City"
)

# Display the map
humidity_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df['Max Temp'] >= 75) &
    (city_data_df['Max Temp'] <= 80) &
    (city_data_df['Humidity'] >= 30) &
    (city_data_df['Humidity'] <= 50)
]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,newman,37.3138,-121.0208,75.15,32,0,25.01,US,1715199586
89,89,albany,42.6001,-73.9662,79.65,48,4,3.00,US,1715199174
175,175,el granada,37.5027,-122.4694,79.92,42,0,19.57,US,1715199593
257,257,colares,38.7992,-9.4469,77.99,38,0,5.75,PT,1715199599
314,314,richfield,44.8833,-93.2830,76.57,35,20,8.05,US,1715199573


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotels_df to store the city, country, coordinates, and humidity
hotel_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotels_df = ideal_weather[hotel_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotels_df['Hotel'] = pd.Series(dtype='object')

# Display sample data
hotels_df.head()


,City,Country,Lat,Lng,Humidity,Hotel
72,newman,US,37.3138,-121.0208,32,NaN
89,albany,US,42.6001,-73.9662,48,NaN
175,el granada,US,37.5027,-122.4694,42,NaN
257,colares,PT,38.7992,-9.4469,38,NaN
314,richfield,US,44.8833,-93.2830,35,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

params = {
    "categories": categories,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in city_data_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
 
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        city_data_df.loc[index, "Hotel"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        city_data_df.loc[index, "Hotel"] = "No hotel found"
        
    # Log the search results
    print(f"{city_data_df.loc[index, 'City']} - nearest hotel: {city_data_df.loc[index, 'Hotel']}")

# Display sample data
city_data_df


Starting hotel search
kerikeri - nearest hotel: Avalon Resort
aden - nearest hotel: فندق قصر الجزيرة
miragoane - nearest hotel: Chez Dem
port-aux-francais - nearest hotel: Keravel
lichinga - nearest hotel: Residencial 2+1
blackmans bay - nearest hotel: Villa Howden
port mathurin - nearest hotel: Escale Vacances
port lincoln - nearest hotel: Boston Hotel
georgetown - nearest hotel: Page 63 hostel
grytviken - nearest hotel: No hotel found
ust-nera - nearest hotel: гостиница "Солнечная"
ebaye - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
solleftea - nearest hotel: Hotell Appelberg
puerto natales - nearest hotel: Dorotea Patagonia Hostel
zaozersk - nearest hotel: Северное сияние
qaqortoq - nearest hotel: Hotel Qaqortoq
peabiru - nearest hotel: No hotel found
kenora - nearest hotel: Travelodge
nuku'alofa - nearest hotel: City Hotel
st. john's - nearest hotel: Alt Hotel
sandnessjoen - nearest hotel: Scandic Syv Søstre
adamstown - nearest hotel: No hotel fo

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,0,kerikeri,-35.2268,173.9474,52.92,84,6,8.99,NZ,1715199581,Avalon Resort
1,1,aden,12.7794,45.0367,86.97,73,92,7.92,YE,1715199581,فندق قصر الجزيرة
2,2,miragoane,18.4423,-73.0876,84.36,68,41,5.91,HT,1715199581,Chez Dem
3,3,port-aux-francais,-49.3500,70.2167,40.44,74,90,18.97,TF,1715199581,Keravel
4,4,lichinga,-13.3128,35.2406,64.49,72,0,9.22,MZ,1715199582,Residencial 2+1
...,...,...,...,...,...,...,...,...,...,...,...
588,588,dokkum,53.3222,5.9970,53.98,86,17,6.44,NL,1715199628,Hotel-Café-Restaurant van der Meer
589,589,yeppoon,-23.1333,150.7333,67.24,81,99,17.45,AU,1715199628,Blossom's on Seaspray
590,590,bulawayo,-20.1500,28.5833,59.49,41,0,8.66,ZW,1715199628,Selboune Hotel
591,591,kuta,-8.7233,115.1723,80.55,94,40,9.22,ID,1715199628,Ida Hotel Bali


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotels_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Hotels",
    geo=True,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Hotel"], 
    xlim=(-180, 180),  
    ylim=(-90, 90),
    tiles='CartoLight',
    color = "City"
)

# Display the map
hotels_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)